## Purpose of this notebook

- Examine similarities and differences between different raters' reliability report grades
- Calculate Cohen's kappa between pairs of raters
- Identify and print reports where raters strongly disagree (grade of 2 vs grade of 0)

## How to use this notebook

- Run each of the cells in order. Make sure you run Cell 01 first.
- Cells 02-03 get the proc_ord_id values unique to each report and the names of the persons who have graded reliability reports.
- Cells 04-07 can be modified to change which users you want to look at. 

In [ ]:
# Cell 01: load libraries
from reliabilityLib import *
from google.cloud import bigquery # SQL table interface on Arcus
import pandas
import numpy
import matplotlib.pyplot as plt

In [ ]:
# Cell 02: Get the list of proc_ord_id values used to identify the reliability reports
procIds = getReliabilityProcOrdIds()

In [ ]:
# Cell 03: Get the distinct users
getDistinctUsersQuery = "select distinct grader_name as graders from lab.grader_table "
getDistinctUsersQuery += "where grade_category = 'Reliability' and grade != 999;"

client = bigquery.Client()
distinctUsers = list(client.query(getDistinctUsersQuery).to_dataframe()['graders'].values)
print(distinctUsers)

In [ ]:
# Cell 04:  Get the reliability reports for the users we want to evaluate (remember indexing in python starts at 0)
meganReports = getReportsForUser(distinctUsers[1], procIds)    
jennaReports = getReportsForUser(distinctUsers[2], procIds)
allyReports = getReportsForUser(distinctUsers[3], procIds)

In [ ]:
# Cell 05: Compare the reliability reports for the users we want to evaluate
kappaMeganJenna = compareUserPairGrades(distinctUsers[1], meganReports, distinctUsers[2], jennaReports)
print("\n\n")
kappaMeganAlly = compareUserPairGrades(distinctUsers[1], meganReports, distinctUsers[3], allyReports)
print("\n\n")
kappaAllyJenna = compareUserPairGrades(distinctUsers[3], allyReports, distinctUsers[2], jennaReports)

In [ ]:
# Cell 06: Identify the proc_ord_ids where each pair of users strongly disagree (one gave a 2, other gave a 0)
disagreementReportsMeganJenna = identifyDisagreementReports(meganReports, jennaReports)
disagreementReportsMeganAlly = identifyDisagreementReports(meganReports, allyReports)
disagreementReportsAllyJenna = identifyDisagreementReports(allyReports, jennaReports)

In [ ]:
# Cell 07: 
# LAUREN - this is the cell where you can look at the disagreement reports for each pair of users
printDisagreementReports(disagreementReportsMeganJenna, meganReports, jennaReports)